In [108]:
from IPython.display import display
import os
import shutil

In [109]:
import numpy as np
import glob
import numpy as np
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [110]:
IMAGE_SIZE = 256
BATCH_SIZE = 8
CHANNELS = 3
EPOCHS = 50

In [111]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "./initial training set", 
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 149 files belonging to 2 classes.


In [112]:
class_names = dataset.class_names
class_names

['mosquito bites on skin', 'tick bites on skin']

In [113]:
len(dataset)

19

In [114]:
train_size = 0.8
train_set_length = len(dataset) * train_size
train_set_length

15.200000000000001

In [115]:
train_ds= dataset.take(16)

In [116]:
len(train_ds)

16

In [117]:
test_ds = dataset.skip(16) 
len(test_ds)

3

In [118]:
val_size = 0.1
len(dataset)*val_size

1.9000000000000001

In [119]:
val_ds = test_ds.take(2)

In [120]:
len(val_ds)

2

In [121]:
test_ds = test_ds.skip(2)
len(test_ds)

1

In [122]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split= 0.1, test_split = 0.1, shuffle=True, shuffle_size =10000):
    ds_size = len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [123]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [124]:
len(train_ds)
len(val_ds)
len(test_ds)

3

In [125]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [126]:
from tensorflow.keras import models, layers

In [127]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [128]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2) 
])

In [78]:
import PIL
import os
from PIL import Image

f = r'./initial training set/mosquito bites on skin'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((128,128))
    img.save(f_img)

f = r'./initial training set/tick bites on skin'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((128,128))
    img.save(f_img)    

In [79]:
from keras import backend as K
# https://www.kaggle.com/xhlulu/severstal-simple-keras-u-net-boilerplate

# COMPETITION METRIC
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [80]:
from tensorflow.keras import backend
def focal_dice(y_true, y_pred,smooth=1e-6):
    """https://github.com/umbertogriffo/focal-loss-keras/blob/master/src/loss_function/losses.py"""
    gamma = 2.0
    alpha = .25

    y_true=backend.flatten(y_true)
    y_pred=backend.flatten(y_pred)

    # clip the prediction value to prevent NaN and Inf
    epsilon = backend.epsilon()
    y_pred = backend.clip(y_pred, epsilon, 1. - epsilon)

    # calculate cross entropy
    cross_entropy = -y_true * backend.log(y_pred)

    # calculate focal loss
    loss = alpha * backend.pow(1 - y_pred, gamma)*cross_entropy

    intersection = backend.sum(y_true*y_pred)  

    dice_loss = 1.0 - (2*intersection + smooth) / (backend.sum(y_true) + backend.sum(y_pred) + smooth)

    return  50*backend.mean(loss)+dice_loss

In [ ]:
train_ds

In [82]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,Callback
sm.set_framework('tf.keras')

# LOAD UNET WITH PRETRAINING FROM IMAGENET
preprocess = get_preprocessing('resnet50') # for resnet, img = (img-110.0)/1.0
model = Unet(input_shape=(128, 128, 3), classes=2, activation='relu')
model.compile(optimizer='adam', loss=focal_dice, metrics=[dice_coef])



#checkpoint to save model
checkpoint = ModelCheckpoint(filepath='cnn.hdf5',monitor='val_dice_coef',verbose=1, save_best_only=True,mode='max')
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_delta=0.00001,mode='min',verbose=0)
callbacks = [checkpoint,rlrp]

tf.cast(train_ds, tf.float32)

history = model.fit(train_ds,  batch_size = BATCH_SIZE,epochs=10,  callbacks=callbacks, verbose=2)


ValueError: Attempt to convert a value (<PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>) to a Tensor.

In [129]:
Input_shape=(BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, CHANNELS)
n_classes = 2
model = models.Sequential([
resize_and_rescale, 
                          data_augmentation,
                          layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE,IMAGE_SIZE)),
                          layers.MaxPooling2D((2,2)),
                          ##########################
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Flatten(),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(n_classes, activation='softmax')
])
model.build(input_shape=Input_shape)

In [130]:
model.summary()
                        

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (8, 256, 256, 3)          0         
                                                                 
 sequential_10 (Sequential)  (8, 256, 256, 3)          0         
                                                                 
 conv2d_12 (Conv2D)          (8, 254, 254, 32)         896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (8, 127, 127, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (8, 125, 125, 64)         18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (8, 62, 62, 64)          0         
 g2D)                                                

In [131]:
model.compile(
    optimizer ='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [27]:
from pathlib import Path
import imghdr

data_dir = "/home/user/datasets/samples/"
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [132]:
history = model.fit(
     train_ds,
     epochs = EPOCHS,
     batch_size = BATCH_SIZE,
     verbose=1,
     validation_data=val_ds
    
)

Epoch 1/50
15/15 [==============================] - 7s 296ms/step - loss: 0.6949 - accuracy: 0.5083 - val_loss: 0.6871 - val_accuracy: 0.5000
Epoch 2/50
15/15 [==============================] - 4s 277ms/step - loss: 0.6975 - accuracy: 0.5250 - val_loss: 0.6856 - val_accuracy: 0.5000
Epoch 3/50
15/15 [==============================] - 4s 284ms/step - loss: 0.6894 - accuracy: 0.5250 - val_loss: 0.6813 - val_accuracy: 0.5000
Epoch 4/50
15/15 [==============================] - 4s 277ms/step - loss: 0.6992 - accuracy: 0.4917 - val_loss: 0.6914 - val_accuracy: 0.6250
Epoch 5/50
15/15 [==============================] - 4s 279ms/step - loss: 0.6913 - accuracy: 0.5417 - val_loss: 0.6854 - val_accuracy: 0.5000
Epoch 6/50
15/15 [==============================] - 4s 284ms/step - loss: 0.6949 - accuracy: 0.5083 - val_loss: 0.6908 - val_accuracy: 0.6250
Epoch 7/50
15/15 [==============================] - 4s 283ms/step - loss: 0.6919 - accuracy: 0.5333 - val_loss: 0.6802 - val_accuracy: 0.5000
Epoch 

15/15 [==============================] - 5s 307ms/step - loss: 0.5778 - accuracy: 0.7250 - val_loss: 0.7084 - val_accuracy: 0.3750
Epoch 42/50
15/15 [==============================] - 4s 293ms/step - loss: 0.6239 - accuracy: 0.6083 - val_loss: 0.7486 - val_accuracy: 0.3750
Epoch 43/50
15/15 [==============================] - 4s 297ms/step - loss: 0.5636 - accuracy: 0.7083 - val_loss: 1.5768 - val_accuracy: 0.5000
Epoch 44/50
15/15 [==============================] - 5s 299ms/step - loss: 0.6231 - accuracy: 0.6250 - val_loss: 0.7631 - val_accuracy: 0.5000
Epoch 45/50
15/15 [==============================] - 4s 297ms/step - loss: 0.5663 - accuracy: 0.6667 - val_loss: 1.0963 - val_accuracy: 0.3750
Epoch 46/50
15/15 [==============================] - 4s 294ms/step - loss: 0.5992 - accuracy: 0.6833 - val_loss: 0.7344 - val_accuracy: 0.3750
Epoch 47/50
15/15 [==============================] - 4s 296ms/step - loss: 0.5568 - accuracy: 0.6750 - val_loss: 0.8096 - val_accuracy: 0.6250
Epoch 48/50

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10))


# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# trainImages = np.asarray(trainImages)
# trainLabels = np.asarray(trainLabels).astype('float32')
# validationLabels = np.asarray(validationLabels).astype('float32')
# validationImages = np.asarray(validationImages)

# history = model.fit(trainImages, trainLabels, epochs=10, 
#                     validation_data=(validationImages, validationLabels))

In [ ]:
# trainImages.shape()
# trainLabels.shape()

# validationImages.shape()
# validationLabels.shape()

In [ ]:
# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
